## Optimization Algorithm: Mini-batch SGD

- $ \theta = \theta - \eta \cdot \nabla J (\theta, x^{i:i+n}, y^{i:i+n}) $ 

$n$은 학습 샘플의 수를 의미합니다. 경사하강법은 미분을 통해 함수의 기울기를 확인하고, 기울기의 반대방향으로 움직이면서 학습하는 방식입니다. 

## Import Packages

In [45]:
'''
torchvision은 이미지처리를 하기 위한 패키지입니다. transforms를 통해 이미지의 사이즈를 조절하거나, tensor로 변환하는 것들이 가능합니다.
'''

import torch 
import torch.nn as nn

from torch.optim.lr_scheduler import StepLR 
import torch.optim as optim 

from torch.utils.data import DataLoader  
import torchvision.transforms as transforms 
import torchvision.datasets as dsets

## Load Dataset

In [46]:
# Set seed 
torch.manual_seed(0)
train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(), 
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=True, 
                           transform=transforms.ToTensor(),     
                           download=True)


batch_size = 100
n_iters = 3000 
num_epochs = n_iters / (len(train_dataset)/batch_size)
num_epochs = int(num_epochs)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

## Build Model

In [54]:
class FFNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FFNN, self).__init__()
        
        self.input_dim = input_dim 
        self.hidden_dim = hidden_dim 
        self.output_dim = output_dim 
        
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        
        self._init_weight()
        
    def forward(self,images):
        out = self.fc1(images)
        out = self.relu(out)
        out = self.fc2(out)
        return out
    
    def _init_weight(self):
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.kaiming_normal_(module.weight)
                nn.init.zeros_(module.bias)

- Momentum
    - $v_t = \gamma u_{t-1} + \eta \cdot \nabla J (\theta - \gamma v_{t-1}, x^{i:i+n}, y^{i:i+n}) $
    - $ \theta = \theta - v_t $ 

- Practical example 
    - Given $\eta_t$ = 0.1, and $\gamma = 0.01$
    - Epoch 0: $\eta_t = 0.1$
    - Epoch 1: $\eta_{t+1} = 0.1(0.1) = 0.01$
    - Epoch 2: $\eta_{t+2} = 0.1(0.1)^2 = 0.001$
    - Epoch n: $\eta_{t+n} = 0.1(0.1)^n$
    

In [56]:
input_dim = 28 * 28
hidden_dim = 100 
output_dim = 10 
learning_rate = 1e-1

model = FFNN(input_dim, hidden_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, nesterov=True)

# step_size를 지정하면 step 마다 learning rate를 gamma만큼 감소시킵니다.
# gamma = decaying factor 
scheduler = StepLR(optimizer, step_size=1, gamma=0.1)

iter = 0 
for epoch in range(num_epochs):

    print(f'Epoch: {epoch}, LR: {scheduler.get_last_lr()}')
    
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        images = images.view(-1, 28*28)
        
        optimizer.zero_grad()
        pred_y = model(images) # model.train()을 사용하지 않으면, images = imgaes.require_grad_()를 설정해주어야 합니다.
        loss = criterion(pred_y, labels) 
        loss.backward()
        optimizer.step()
        
        iter += 1 
        
        if iter % 500 == 0 :
            correct = 0
            total = 0 
            model.eval()
            for images, labels in test_loader:
                images = images.view(-1, 28*28)
                
                pred_y = model(images)
                pred_y = torch.argmax(pred_y.data, 1)
                
                total += labels.size(0)
                
                correct += (pred_y == labels).sum()
                
            accuracy = 100 * correct / total 
        
            print(f'Iteration: {iter}, Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}%')
    scheduler.step()    

                

Epoch: 0, LR: [0.1]
Iteration: 500, Loss: 0.2473, Accuracy: 96.47%
Epoch: 1, LR: [0.010000000000000002]
Iteration: 1000, Loss: 0.0970, Accuracy: 97.59%
Epoch: 2, LR: [0.0010000000000000002]
Iteration: 1500, Loss: 0.0247, Accuracy: 97.75%
Epoch: 3, LR: [0.00010000000000000003]
Iteration: 2000, Loss: 0.0711, Accuracy: 97.82%
Epoch: 4, LR: [1.0000000000000004e-05]
Iteration: 2500, Loss: 0.0305, Accuracy: 97.82%
Iteration: 3000, Loss: 0.1096, Accuracy: 97.82%


# Code for step-wise learning rate decay at every 2 epoch 

In [57]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets

# Set seed
torch.manual_seed(0)

# Where to add a new import
from torch.optim.lr_scheduler import StepLR

'''
STEP 1: LOADING DATASET
'''

train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

'''
STEP 2: MAKING DATASET ITERABLE
'''

batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

'''
STEP 3: CREATE MODEL CLASS
'''
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Linear function
        self.fc1 = nn.Linear(input_dim, hidden_dim) 
        # Non-linearity
        self.relu = nn.ReLU()
        # Linear function (readout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)  
        self._init_weight()
            
    def forward(self, x):
        # Linear function
        out = self.fc1(x)
        # Non-linearity
        out = self.relu(out)
        # Linear function (readout)
        out = self.fc2(out)
        return out

    def _init_weight(self):
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.kaiming_normal_(module.weight)
                nn.init.zeros_(module.bias)

'''
STEP 4: INSTANTIATE MODEL CLASS
'''
input_dim = 28*28
hidden_dim = 100
output_dim = 10

model = FeedforwardNeuralNetModel(input_dim, hidden_dim, output_dim)

'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss()

'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
learning_rate = 0.1

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, nesterov=True)

'''
STEP 7: INSTANTIATE STEP LEARNING SCHEDULER CLASS
'''
# step_size: at how many multiples of epoch you decay
# step_size = 1, after every 2 epoch, new_lr = lr*gamma 
# step_size = 2, after every 2 epoch, new_lr = lr*gamma 

# gamma = decaying factor
scheduler = StepLR(optimizer, step_size=2, gamma=0.1)

'''
STEP 7: TRAIN THE MODEL
'''
iter = 0
for epoch in range(num_epochs):

    # Print Learning Rate
    print('Epoch:', epoch,'LR:', scheduler.get_last_lr())
    for i, (images, labels) in enumerate(train_loader):
        # Load images as Variable
        images = images.view(-1, 28*28).requires_grad_()

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = model(images)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        
        iter += 1

        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                # Load images to a Torch Variable
                images = images.view(-1, 28*28).requires_grad_()

                # Forward pass only to get logits/output
                outputs = model(images)

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)

                # Total number of labels
                total += labels.size(0)

                # Total correct predictions
                correct += (predicted == labels).sum()

            accuracy = 100 * correct / total

            # Print Loss
            print('Iteration: {}. Loss: {:.4f}. Accuracy: {:.2f}%'.format(iter, loss.item(), accuracy))
    # Decay Learning Rate
    scheduler.step()
    

Epoch: 0 LR: [0.1]
Iteration: 500. Loss: 0.1836. Accuracy: 95.85%
Epoch: 1 LR: [0.1]
Iteration: 1000. Loss: 0.1169. Accuracy: 96.65%
Epoch: 2 LR: [0.010000000000000002]
Iteration: 1500. Loss: 0.0161. Accuracy: 97.59%
Epoch: 3 LR: [0.010000000000000002]
Iteration: 2000. Loss: 0.0467. Accuracy: 97.52%
Epoch: 4 LR: [0.0010000000000000002]
Iteration: 2500. Loss: 0.0202. Accuracy: 97.68%
Iteration: 3000. Loss: 0.0766. Accuracy: 97.69%


# Code for step-wise learning rate decay at every epoch with larger gamma 

In [58]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets

# Set seed
torch.manual_seed(0)

# Where to add a new import
from torch.optim.lr_scheduler import StepLR

'''
STEP 1: LOADING DATASET
'''

train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

'''
STEP 2: MAKING DATASET ITERABLE
'''

batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

'''
STEP 3: CREATE MODEL CLASS
'''
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Linear function
        self.fc1 = nn.Linear(input_dim, hidden_dim) 
        # Non-linearity
        self.relu = nn.ReLU()
        # Linear function (readout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)  

    def forward(self, x):
        # Linear function
        out = self.fc1(x)
        # Non-linearity
        out = self.relu(out)
        # Linear function (readout)
        out = self.fc2(out)
        return out
'''
STEP 4: INSTANTIATE MODEL CLASS
'''
input_dim = 28*28
hidden_dim = 100
output_dim = 10

model = FeedforwardNeuralNetModel(input_dim, hidden_dim, output_dim)

'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss()


'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
learning_rate = 0.1

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, nesterov=True)

'''
STEP 7: INSTANTIATE STEP LEARNING SCHEDULER CLASS
'''
# step_size: at how many multiples of epoch you decay
# step_size = 1, after every 2 epoch, new_lr = lr*gamma 
# step_size = 2, after every 2 epoch, new_lr = lr*gamma 

# gamma = decaying factor
scheduler = StepLR(optimizer, step_size=2, gamma=0.96)

'''
STEP 7: TRAIN THE MODEL
'''
iter = 0
for epoch in range(num_epochs):

    # Print Learning Rate
    print('Epoch:', epoch,'LR:', scheduler.get_last_lr())
    for i, (images, labels) in enumerate(train_loader):
        # Load images as Variable
        images = images.view(-1, 28*28).requires_grad_()

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = model(images)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                # Load images to a Torch Variable
                images = images.view(-1, 28*28)

                # Forward pass only to get logits/output
                outputs = model(images)

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)

                # Total number of labels
                total += labels.size(0)

                # Total correct predictions
                correct += (predicted == labels).sum()

            accuracy = 100 * correct / total

            # Print Loss
            print('Iteration: {}. Loss: {:.4f}. Accuracy: {:.2f}%'.format(iter, loss.item(), accuracy))
            
    # Decay Learning Rate
    scheduler.step()

Epoch: 0 LR: [0.1]
Iteration: 500. Loss: 0.0820. Accuracy: 96.00%
Epoch: 1 LR: [0.1]
Iteration: 1000. Loss: 0.1061. Accuracy: 96.39%
Epoch: 2 LR: [0.096]
Iteration: 1500. Loss: 0.1027. Accuracy: 96.81%
Epoch: 3 LR: [0.096]
Iteration: 2000. Loss: 0.0377. Accuracy: 97.38%
Epoch: 4 LR: [0.09215999999999999]
Iteration: 2500. Loss: 0.0255. Accuracy: 97.58%
Iteration: 3000. Loss: 0.1554. Accuracy: 97.78%


# Code for reduce on loss plateau learning rate decay of factor 0.1 and 0 patience

In [60]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets

# Set seed
torch.manual_seed(0)

# Where to add a new import
from torch.optim.lr_scheduler import ReduceLROnPlateau

'''
STEP 1: LOADING DATASET
'''

train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

'''
STEP 2: MAKING DATASET ITERABLE
'''

batch_size = 100
n_iters = 6000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

'''
STEP 3: CREATE MODEL CLASS
'''
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Linear function
        self.fc1 = nn.Linear(input_dim, hidden_dim) 
        # Non-linearity
        self.relu = nn.ReLU()
        # Linear function (readout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)  

    def forward(self, x):
        # Linear function
        out = self.fc1(x)
        # Non-linearity
        out = self.relu(out)
        # Linear function (readout)
        out = self.fc2(out)
        return out
'''
STEP 4: INSTANTIATE MODEL CLASS
'''
input_dim = 28*28
hidden_dim = 100
output_dim = 10

model = FeedforwardNeuralNetModel(input_dim, hidden_dim, output_dim)

'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss()


'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
learning_rate = 0.1

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, nesterov=True)

'''
STEP 7: INSTANTIATE STEP LEARNING SCHEDULER CLASS
'''
# lr = lr * factor 
# mode='max': look for the maximum validation accuracy to track
# patience: number of epochs - 1 where loss plateaus before decreasing LR
        # patience = 0, after 1 bad epoch, reduce LR
# factor = decaying factor
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=0, verbose=True)

'''
STEP 7: TRAIN THE MODEL
'''
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Load images as Variable
        images = images.view(-1, 28*28).requires_grad_()

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = model(images)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                # Load images to a Torch Variable
                images = images.view(-1, 28*28)

                # Forward pass only to get logits/output
                outputs = model(images)

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)

                # Total number of labels
                total += labels.size(0)

                # Total correct predictions
                # Without .item(), it is a uint8 tensor which will not work when you pass this number to the scheduler
                correct += (predicted == labels).sum().item()

            accuracy = 100 * correct / total

            # Print Loss
            # print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.data[0], accuracy))

    # Decay Learning Rate, pass validation accuracy for tracking at every epoch
    print('Epoch {} completed'.format(epoch))
    print('Loss: {:.4f}. Accuracy: {:.2f}%'.format(loss.item(), accuracy))
    print('-'*20)
    scheduler.step(accuracy)

Epoch 0 completed
Loss: 0.1791. Accuracy: 96.00%
--------------------
Epoch 1 completed
Loss: 0.0967. Accuracy: 96.39%
--------------------
Epoch 2 completed
Loss: 0.0164. Accuracy: 96.75%
--------------------
Epoch 3 completed
Loss: 0.0771. Accuracy: 97.38%
--------------------
Epoch 4 completed
Loss: 0.1176. Accuracy: 97.67%
--------------------
Epoch 5 completed
Loss: 0.0780. Accuracy: 97.53%
--------------------
Epoch 00006: reducing learning rate of group 0 to 1.0000e-02.
Epoch 6 completed
Loss: 0.0079. Accuracy: 98.08%
--------------------
Epoch 7 completed
Loss: 0.0104. Accuracy: 98.10%
--------------------
Epoch 8 completed
Loss: 0.0089. Accuracy: 98.12%
--------------------
Epoch 9 completed
Loss: 0.0060. Accuracy: 98.16%
--------------------


# Code for reduce on loss plateau learning rate decay with factor 0.5 and 0 patience 

In [61]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets

# Set seed
torch.manual_seed(0)

# Where to add a new import
from torch.optim.lr_scheduler import ReduceLROnPlateau

'''
STEP 1: LOADING DATASET
'''

train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

'''
STEP 2: MAKING DATASET ITERABLE
'''

batch_size = 100
n_iters = 6000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

'''
STEP 3: CREATE MODEL CLASS
'''
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Linear function
        self.fc1 = nn.Linear(input_dim, hidden_dim) 
        # Non-linearity
        self.relu = nn.ReLU()
        # Linear function (readout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)  

    def forward(self, x):
        # Linear function
        out = self.fc1(x)
        # Non-linearity
        out = self.relu(out)
        # Linear function (readout)
        out = self.fc2(out)
        return out
'''
STEP 4: INSTANTIATE MODEL CLASS
'''
input_dim = 28*28
hidden_dim = 100
output_dim = 10

model = FeedforwardNeuralNetModel(input_dim, hidden_dim, output_dim)

'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss()


'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
learning_rate = 0.1

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, nesterov=True)

'''
STEP 7: INSTANTIATE STEP LEARNING SCHEDULER CLASS
'''
# lr = lr * factor 
# mode='max': look for the maximum validation accuracy to track
# patience: number of epochs - 1 where loss plateaus before decreasing LR
        # patience = 0, after 1 bad epoch, reduce LR
# factor = decaying factor
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=0, verbose=True)

'''
STEP 7: TRAIN THE MODEL
'''
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Load images as Variable
        images = images.view(-1, 28*28).requires_grad_()

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = model(images)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                # Load images to a Torch Variable
                images = images.view(-1, 28*28)

                # Forward pass only to get logits/output
                outputs = model(images)

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)

                # Total number of labels
                total += labels.size(0)

                # Total correct predictions
                 # Without .item(), it is a uint8 tensor which will not work when you pass this number to the scheduler
                correct += (predicted == labels).sum().item()

            accuracy = 100 * correct / total

            # Print Loss
            # print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.data[0], accuracy))

    # Decay Learning Rate, pass validation accuracy for tracking at every epoch
    print('Epoch {} completed'.format(epoch))
    print('Loss: {:.4f}. Accuracy: {:.2f}%'.format(loss.item(), accuracy))
    print('-'*20)
    scheduler.step(accuracy)

Epoch 0 completed
Loss: 0.1791. Accuracy: 96.00%
--------------------
Epoch 1 completed
Loss: 0.0967. Accuracy: 96.39%
--------------------
Epoch 2 completed
Loss: 0.0164. Accuracy: 96.75%
--------------------
Epoch 3 completed
Loss: 0.0771. Accuracy: 97.38%
--------------------
Epoch 4 completed
Loss: 0.1176. Accuracy: 97.67%
--------------------
Epoch 5 completed
Loss: 0.0780. Accuracy: 97.53%
--------------------
Epoch 00006: reducing learning rate of group 0 to 5.0000e-02.
Epoch 6 completed
Loss: 0.0046. Accuracy: 98.01%
--------------------
Epoch 7 completed
Loss: 0.0106. Accuracy: 97.99%
--------------------
Epoch 00008: reducing learning rate of group 0 to 2.5000e-02.
Epoch 8 completed
Loss: 0.0124. Accuracy: 98.19%
--------------------
Epoch 9 completed
Loss: 0.0050. Accuracy: 98.17%
--------------------
Epoch 00010: reducing learning rate of group 0 to 1.2500e-02.
